# To do

- climate zone join
- egrid join

In [1]:
from usagrid import s3
import pandas as pd
import numpy as np
from pathlib import Path
import geopandas as gpd


pd.options.display.float_format = '{:20,.2f}'.format

In [2]:
from tqdm import tqdm

paths = s3.list_files_in_folder("usagrid","bronze/powerplants/")[:]

files = [p for p in paths[:] if Path(p).suffix]

pbar =  tqdm(files,leave=True)

for f in pbar:

    pbar.set_description("Processing: " + f)

    read_path = Path(f)

    df_temp = s3.read_pyarrow_df_from_s3("usagrid",f).to_pandas()



Processing: bronze/powerplants/powperplants_concat.arrow: 100%|██████████| 1/1 [00:00<00:00,  1.56it/s]


In [3]:
powerplant_shape = f's3://usagrid/mappings/Power_Plants.zip'

pp_shape = gpd.read_file(powerplant_shape)

In [4]:

pp_shape = pp_shape.assign(plantCode=pp_shape.Plant_Code.astype(str),
                            plantName=pp_shape.Plant_Name)

In [5]:
power_plant = pp_shape[["plantCode","plantName","Utility_ID","geometry"]].merge(df_temp,on=["plantCode","plantName"])

In [6]:
ba_shape_f = f's3://usagrid/mappings/BalancingAuthorties.zip'

ba_shape = gpd.read_file(ba_shape_f)

In [145]:
def spatial_join(df1:pd.DataFrame,
                df2:pd.DataFrame,
                predicate:str="within",
                how:str="left") -> gpd.GeoDataFrame:

    if df1.crs != df2.crs:

        df2 = df2.to_crs(df1.crs)


    return df1.sjoin(df2,predicate=predicate,how=how)

In [136]:
unique_pp = power_plant.drop_duplicates(subset=["plantCode"])

In [137]:
pp_ba = unique_pp[["plantCode","plantName","Utility_ID","geometry"]]\
    .pipe(spatial_join,ba_shape[["NAME","geometry"]],how="left")

In [138]:
pp_ba_nafilled = pp_ba.fillna(value={"NAME":""})

In [139]:
pp_ba.NAME.apply(lambda x: type(x)).value_counts()

NAME
<class 'str'>      14064
<class 'float'>      225
Name: count, dtype: int64

In [140]:
pp_ba_nafilled.NAME.apply(lambda x: type(x)).value_counts()

NAME
<class 'str'>    14289
Name: count, dtype: int64

# Deal with multiple balancing authorities being mapped
- this is because of redundancy in balancing authority regions

In [141]:
names_collapse = pp_ba_nafilled.groupby(['plantCode',"plantName"]).agg({"NAME":"unique"}).reset_index()\
                .drop_duplicates(subset=["plantCode","plantName"])\
                .rename(columns={"NAME":"balancing_authority_list"})

names_count = pp_ba_nafilled.groupby(['plantCode',"plantName"]).agg({"NAME":"nunique"}).reset_index()\
                .drop_duplicates(subset=["plantCode","plantName"])\
                .rename(columns={"NAME":"balancing_authority_count"})

In [142]:
ba_summary = unique_pp.merge(names_collapse,on=["plantCode","plantName"],how="left")\
        .merge(names_count,on=["plantCode","plantName"],how="left")

In [143]:
ba_summary

,plantCode,plantName,Utility_ID,geometry,period,fuel2002,fuelTypeDescription,state,stateDescription,primeMover,...,total-consumption,total-consumption-units,total-consumption-btu,consumption-for-eg-btu-units,generation-units,gross-generation-units,total-consumption-btu-units,year,balancing_authority_list,balancing_authority_count
0,1,Sand Point,63560,POINT (-160.49722 55.33972),2022-03-01,WND,Wind,AK,Alaska,WT,...,0.00,None,156.00,MMBtu,megawatthours,megawatthours,MMBtu,2022,[],1
1,2,Bankhead Dam,195,POINT (-87.35682 33.45867),2022-03-01,WAT,Hydroelectric Conventional,AL,Alabama,HY,...,0.00,None,"55,852.00",MMBtu,megawatthours,megawatthours,MMBtu,2022,"[SOUTHERN COMPANY SERVICES, INC. - TRANS]",1
2,3,Barry,195,POINT (-88.01030 31.00690),2022-03-01,SUB,Coal,AL,Alabama,ST,...,0.00,MMBtu per short tons,0.00,MMBtu,megawatthours,megawatthours,MMBtu,2022,"[SOUTHERN COMPANY SERVICES, INC. - TRANS, MIDC...",2
3,4,Walter Bouldin Dam,195,POINT (-86.28306 32.58389),2022-03-01,WAT,Hydroelectric Conventional,AL,Alabama,HY,...,0.00,None,"162,772.00",MMBtu,megawatthours,megawatthours,MMBtu,2022,"[POWERSOUTH ENERGY COOPERATIVE, SOUTHERN COMPA...",2
4,9,Copper,5701,POINT (-106.37500 31.75690),2022-03-01,NG,Natural Gas,TX,Texas,GT,...,"7,875.00",MMBtu per Mcf,"8,048.00",MMBtu,megawatthours,megawatthours,MMBtu,2022,[EL PASO ELECTRIC COMPANY],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11041,66700,Banner Solar,65702,POINT (-78.55542 35.41372),2022-11-01,SUN,Solar,NC,North Carolina,PV,...,0.00,None,"2,092.00",MMBtu,megawatthours,megawatthours,MMBtu,2022,[DUKE ENERGY PROGRESS EAST],1
11042,66709,Beulah Solar,65723,POINT (-81.57116 33.94521),2022-11-01,SUN,Solar,SC,South Carolina,PV,...,0.00,None,461.00,MMBtu,megawatthours,megawatthours,MMBtu,2022,"[DUKE ENERGY CAROLINAS, SOUTH CAROLINA ELECTRI...",2
11043,66710,Brick City Solar,65724,POINT (-77.98345 34.72640),2022-11-01,SUN,Solar,NC,North Carolina,PV,...,0.00,None,"1,703.00",MMBtu,megawatthours,megawatthours,MMBtu,2022,[DUKE ENERGY PROGRESS EAST],1
11044,66718,ESA Albemarle NC Solar,65731,POINT (-80.12365 35.39606),2022-07-01,SUN,Solar,NC,North Carolina,PV,...,0.00,None,"1,549.00",MMBtu,megawatthours,megawatthours,MMBtu,2022,"[DUKE ENERGY PROGRESS EAST, DUKE ENERGY CAROLI...",2


In [144]:
# ba_summary.select_dtypes(include=["float"]).div(ba_summary["balancing_authority_count"],axis=0)

ba_summary[["balancing_authority_count"]]

,balancing_authority_count
0,1
1,1
2,2
3,2
4,1
...,...
11041,1
11042,2
11043,1
11044,2
